## Cleaning

In [284]:
import pandas as pd
import numpy as np
import math
import re
from datetime import datetime
import time

In [147]:
df = pd.read_csv('House_Votes.csv', encoding = 'unicode_escape')

In [148]:
df['dnum'] = df['district'].apply(lambda x: '{0:0>2}'.format(x))

In [149]:
df['dnum'] = df.dnum.apply(str)

In [150]:
df['year'] = df.year.apply(str)

In [151]:
df['district_id'] = df[['state_po', 'dnum']].apply(lambda x: '-'.join(x), axis=1)

In [152]:
df['yr_district_id'] = df[['year', 'district_id']].apply(lambda x: '-'.join(x), axis=1)

In [153]:
df.drop(['state_po', 'state_fips', 'state_cen', 'state_ic',
       'office', 'district', 'mode', 'unofficial', 'version', 'dnum'], axis = 1, inplace = True)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29636 entries, 0 to 29635
Data columns (total 12 columns):
year              29636 non-null object
state             29636 non-null object
stage             29606 non-null object
runoff            20980 non-null object
special           29636 non-null bool
candidate         27515 non-null object
party             26208 non-null object
writein           29636 non-null bool
candidatevotes    29636 non-null int64
totalvotes        29636 non-null int64
district_id       29636 non-null object
yr_district_id    29636 non-null object
dtypes: bool(2), int64(2), object(8)
memory usage: 2.3+ MB


In [155]:
df = df[(df.stage != 'pri') & (df.runoff != True) & (df.special != True)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29538 entries, 0 to 29635
Data columns (total 12 columns):
year              29538 non-null object
state             29538 non-null object
stage             29538 non-null object
runoff            20912 non-null object
special           29538 non-null bool
candidate         27422 non-null object
party             26115 non-null object
writein           29538 non-null bool
candidatevotes    29538 non-null int64
totalvotes        29538 non-null int64
district_id       29538 non-null object
yr_district_id    29538 non-null object
dtypes: bool(2), int64(2), object(8)
memory usage: 2.5+ MB


In [156]:
df.year.value_counts()

2000    1608
1996    1594
1992    1524
2010    1500
2016    1440
2002    1431
2012    1427
2008    1381
1998    1373
2018    1359
2014    1357
2006    1344
2004    1340
1976    1306
1994    1301
1980    1285
1982    1275
1984    1171
1990    1155
1978    1130
1988    1129
1986    1108
Name: year, dtype: int64

In [142]:
22*435

9570

In [36]:
df.party.unique()

array(['democrat', 'republican', nan, 'prohibition', 'national democrat',
       'independent', 'libertarian', 'peace and freedom',
       'american independent', 'u.s. labor', 'socialist workers',
       'american', 'la raza unida', 'indpendent', 'communist',
       'conservative', 'socialist labor',
       'independents for godly government', "people's", 'workers',
       'white power', 'human rights', 'independent american',
       'new majority', 'labor', 'regular democracy', 'pro-life',
       'restoration', 'individual needs center',
       "independent taxpayer's watchdog", 'politicians are crooks',
       'jobs, equality, peace', 'consumer action', 'bring us together',
       'individual americans independence', 'silent majority',
       "people's independent", 'liberal', 'mayflower', 'coequal citizens',
       'revolutionary workers', 'independent conservatives',
       'constitution', 'citizens for haas', 'united states labor',
       'aloha democratic', 'socialist', "worker'

In [64]:
df.party.value_counts()


democrat                                      9060
republican                                    8839
libertarian                                   2564
independent                                   1127
conservative                                   621
green                                          491
natural law                                    371
liberal                                        266
working families                               245
independence                                   229
right to life                                  225
reform                                         173
peace and freedom                              165
constitution                                   140
socialist workers                              132
american independent                           104
democratic-farmer-labor                        101
american                                        72
none                                            67
u.s. taxpayers                 

In [157]:
df.head()

,year,state,stage,runoff,special,candidate,party,writein,candidatevotes,totalvotes,district_id,yr_district_id
0,1976,Alabama,gen,False,False,Bill Davenport,democrat,False,58906,157170,AL-01,1976-AL-01
1,1976,Alabama,gen,False,False,Jack Edwards,republican,False,98257,157170,AL-01,1976-AL-01
2,1976,Alabama,gen,False,False,NaN,NaN,True,7,157170,AL-01,1976-AL-01
3,1976,Alabama,gen,False,False,J. Carole Keahey,democrat,False,66288,156362,AL-02,1976-AL-02
4,1976,Alabama,gen,False,False,NaN,NaN,True,5,156362,AL-02,1976-AL-02


In [93]:
df['year'] = df.year.apply(int)

In [158]:
def func(x):
     return pd.Series(dict(candidatevotes = x['candidatevotes'].max(), 
                           totalvotes = x['candidatevotes'].sum()))

In [159]:
winner = pd.DataFrame(df.groupby(['yr_district_id']).apply(func))

In [160]:
winner.reset_index(inplace=True)

In [161]:
winner

,yr_district_id,candidatevotes,totalvotes
0,1976-AK-00,83722,118208
1,1976-AL-01,98257,157170
2,1976-AL-02,90069,156362
3,1976-AL-03,106935,108048
4,1976-AL-04,141490,176022
5,1976-AL-05,113553,113560
6,1976-AL-06,92113,162518
7,1976-AL-07,110496,110501
8,1976-AR-01,116217,168782
9,1976-AR-02,144780,167607


In [188]:
winner_df = pd.merge(winner, df[['year', 'state', 'candidate', 'party', 'writein',
       'candidatevotes', 'district_id', 'yr_district_id']], on=['yr_district_id', 'candidatevotes'])
winner_df

,yr_district_id,candidatevotes,totalvotes,year,state,candidate,party,writein,district_id
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02


In [189]:
winner_df['voteshare'] = winner_df.candidatevotes / winner_df.totalvotes

In [190]:
winner_df.voteshare.describe()

count    9552.000000
mean        0.670675
std         0.134570
min         0.274997
25%         0.573844
50%         0.648718
75%         0.731232
max         1.000000
Name: voteshare, dtype: float64

In [191]:
len(winner_df[winner_df.voteshare < 0.5])

391

In [192]:
winner_df.loc[winner_df['voteshare'] < 0.275]


,yr_district_id,candidatevotes,totalvotes,year,state,candidate,party,writein,district_id,voteshare
8858,2016-LA-03,77671,282443,2016,Louisiana,Clay Higgins,republican,False,LA-03,0.274997


In [193]:
winner_df.columns

Index(['yr_district_id', 'candidatevotes', 'totalvotes', 'year', 'state',
       'candidate', 'party', 'writein', 'district_id', 'voteshare'],
      dtype='object')

In [219]:
winner_df.columns = ['yr_district_id', 'winner_votes', 'totalvotes', 'year', 'state',
       'winner', 'winner_party', 'writein', 'district_id', 'winner_voteshare']

In [220]:
winner_df.head()

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820


In [197]:
winner_df.loc[winner_df["winner_party"] == 'independent']

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,voteshare
3457,1990-VT-00,117522,209856,1990,Vermont,Bernie Sanders,independent,False,VT-00,0.560013
3892,1992-VT-00,162724,281162,1992,Vermont,Bernard Sanders,independent,False,VT-00,0.578755
4327,1994-VT-00,105502,211449,1994,Vermont,Bernard Sanders,independent,False,VT-00,0.498948
4574,1996-MO-08,112472,222854,1996,Missouri,Jo Ann Emerson,independent,False,MO-08,0.504689
4749,1996-VT-00,140678,254706,1996,Vermont,Bernard Sanders,independent,False,VT-00,0.552315
5184,1998-VT-00,136403,215133,1998,Vermont,Bernard Sanders,independent,False,VT-00,0.634040
5612,2000-VA-05,143312,212705,2000,Virginia,"Virgil H. Goode, Jr.",independent,False,VA-05,0.673759
5619,2000-VT-00,196118,283366,2000,Vermont,Bernard Sanders,independent,False,VT-00,0.692101
6055,2002-VT-00,144880,225255,2002,Vermont,Bernard Sanders,independent,False,VT-00,0.643182
6490,2004-VT-00,205774,305008,2004,Vermont,Bernard Sanders,independent,False,VT-00,0.674651


In [198]:
winner_df["winner_party"]= winner_df["winner_party"].replace("democratic-farmer-labor", "democrat") 

In [199]:
winner_df["winner_party"]= winner_df["winner_party"].replace("foglietta (democrat)", "democrat") 

In [200]:
winner_df["winner_party"]= winner_df["winner_party"].replace("independent-republican", "republican") 

In [201]:
winner_df.loc[winner_df["winner"] == 'Bernie Sanders', 'winner_party'] = 'democrat'
winner_df.loc[winner_df["winner"] == 'Bernard Sanders', 'winner_party'] = 'democrat'
winner_df.loc[winner_df["winner"] == 'Jo Ann Emerson', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Virgil H. Goode, Jr.', 'winner_party'] = 'republican'

In [217]:
winner_df.winner_party.value_counts()

democrat      5144
republican    4405
Name: winner_party, dtype: int64

In [239]:
winner_df[winner_df.isna().any(axis=1)]

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare


In [234]:
winner_df.loc[winner_df["winner"] == 'Thomas G. Tancredo', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Mo Brooks', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Christopher H. Smith', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Mike Gallagher', 'winner_party'] = 'republican'

In [238]:
winner_df.loc[winner_df["winner"] == 'Frederica S. Wilson', 'winner_votes'] = 1
winner_df.loc[winner_df["winner"] == 'Frederica S. Wilson', 'totalvotes'] = 1
winner_df.loc[winner_df["winner"] == 'Frederica S. Wilson', 'winner_voteshare'] = 1

In [206]:
df1.party.value_counts()

democrat                                      9026
republican                                    8801
libertarian                                   2559
independent                                   1115
conservative                                   620
green                                          491
natural law                                    371
liberal                                        266
working families                               245
independence                                   229
right to life                                  225
reform                                         173
peace and freedom                              165
constitution                                   140
socialist workers                              131
american independent                           104
democratic-farmer-labor                        101
american                                        72
none                                            67
no party affiliation           

In [205]:
df1 = df.copy()

In [207]:
df1["party"]= df1["party"].replace("green", "democrat") 

In [208]:
df1["party"]= df1["party"].replace("liberal", "democrat") 

In [209]:
df1["party"]= df1["party"].replace("democratic-farmer-labor", "democrat") 

In [210]:
df1["party"]= df1["party"].replace("socialist workers", "democrat") 

In [211]:
df1["party"]= df1["party"].replace("foglietta (democrat)", "democrat") 

In [212]:
df1["party"]= df1["party"].replace("independent-republican", "republican") 

In [213]:
df1["party"]= df1["party"].replace("libertarian", "republican") 

In [214]:
df1["party"]= df1["party"].replace("conservative", "republican") 

In [215]:
df1.loc[df1["candidate"] == 'Bernie Sanders', 'party'] = 'democrat'
df1.loc[df1["candidate"] == 'Bernard Sanders', 'party'] = 'democrat'
df1.loc[df1["candidate"] == 'Jo Ann Emerson', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Virgil H. Goode, Jr.', 'party'] = 'republican'

In [250]:
df1.loc[df1["candidate"] == 'Thomas G. Tancredo', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Mo Brooks', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Christopher H. Smith', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Mike Gallagher', 'party'] = 'republican'

In [251]:
df1.loc[df1["candidate"] == 'Frederica S. Wilson', 'candidatevotes'] = 1
df1.loc[df1["candidate"] == 'Frederica S. Wilson', 'totalvotes'] = 1

In [216]:
df1.party.value_counts()

republican                                    11991
democrat                                      10024
independent                                    1104
natural law                                     371
working families                                245
independence                                    229
right to life                                   225
reform                                          173
peace and freedom                               165
constitution                                    140
american independent                            104
american                                         72
none                                             67
no party affiliation                             63
u.s. taxpayers                                   63
independent american                             47
u.s. labor                                       36
women's equality                                 32
other                                            26
citizens    

In [218]:
df1.columns

Index(['year', 'state', 'stage', 'runoff', 'special', 'candidate', 'party',
       'writein', 'candidatevotes', 'totalvotes', 'district_id',
       'yr_district_id'],
      dtype='object')

In [221]:
dem_vote_df = df1[df1.party == 'democrat']

In [223]:
len(dem_vote_df)

10024

In [224]:
def func(x):
     return pd.Series(dict(dem_votes = x['candidatevotes'].sum(), 
                           dem_candidate = x['candidate'].unique()[0]))

In [225]:
dem_vote = pd.DataFrame(dem_vote_df.groupby(['yr_district_id']).apply(func))

In [226]:
dem_vote.reset_index(inplace=True)

In [227]:
len(dem_vote)

9079

In [228]:
dem_vote.head()

,yr_district_id,dem_votes,dem_candidate
0,1976-AK-00,34194,Eben Hopson
1,1976-AL-01,58906,Bill Davenport
2,1976-AL-02,66288,J. Carole Keahey
3,1976-AL-03,106935,Bill Nichols
4,1976-AL-04,141490,Tom Bevill


In [232]:
winner_df

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05,0.999938
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06,0.566786
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07,0.999955
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01,0.688563
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02,0.863806


In [240]:
wd_df = pd.merge(winner_df, dem_vote, on=['yr_district_id'], how='left')
wd_df

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare,dem_votes,dem_candidate
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260,34194.0,Eben Hopson
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164,58906.0,Bill Davenport
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029,66288.0,J. Carole Keahey
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699,106935.0,Bill Nichols
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820,141490.0,Tom Bevill
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05,0.999938,113553.0,Ronnie G. Flippo
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06,0.566786,69384.0,Mel Bailey
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07,0.999955,110496.0,Walter Flowers
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01,0.688563,116217.0,Bill Alexander
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02,0.863806,144780.0,Jim Guy Tucker


In [252]:
rep_vote_df = df1[df1.party == 'republican']

In [253]:
len(rep_vote_df)

11995

In [254]:
def rfunc(x):
     return pd.Series(dict(rep_votes = x['candidatevotes'].sum(), 
                           rep_candidate = x['candidate'].unique()[0]))

In [255]:
rep_vote = pd.DataFrame(rep_vote_df.groupby(['yr_district_id']).apply(rfunc))

In [256]:
rep_vote.reset_index(inplace=True)

In [257]:
len(rep_vote)

8931

In [258]:
rep_vote.head()

,yr_district_id,rep_votes,rep_candidate
0,1976-AK-00,83722,Don Young
1,1976-AL-01,98257,Jack Edwards
2,1976-AL-02,90069,"William L. """"Bill"""" Dickinson"
3,1976-AL-04,34531,Leonard Wilson
4,1976-AL-06,92113,"John H. Buchanan, Jr."


In [259]:
votes_df = pd.merge(wd_df, rep_vote, on=['yr_district_id'], how='left')
votes_df

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare,dem_votes,dem_candidate,rep_votes,rep_candidate
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260,34194.0,Eben Hopson,83722.0,Don Young
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164,58906.0,Bill Davenport,98257.0,Jack Edwards
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029,66288.0,J. Carole Keahey,90069.0,"William L. """"Bill"""" Dickinson"
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699,106935.0,Bill Nichols,NaN,NaN
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820,141490.0,Tom Bevill,34531.0,Leonard Wilson
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05,0.999938,113553.0,Ronnie G. Flippo,NaN,NaN
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06,0.566786,69384.0,Mel Bailey,92113.0,"John H. Buchanan, Jr."
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07,0.999955,110496.0,Walter Flowers,NaN,NaN
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01,0.688563,116217.0,Bill Alexander,52565.0,"Harlan """"Bo"""" Holleman"
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02,0.863806,144780.0,Jim Guy Tucker,22819.0,James J. Kelly


In [261]:
votes_df['dem_voteshare'] = votes_df.dem_votes/ votes_df.totalvotes

In [262]:
votes_df['rep_voteshare'] = votes_df.rep_votes/ votes_df.totalvotes

In [265]:
votes_df.head()

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare,dem_votes,dem_candidate,rep_votes,rep_candidate,dem_voteshare,rep_voteshare
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260,34194.0,Eben Hopson,83722.0,Don Young,0.289270,0.708260
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164,58906.0,Bill Davenport,98257.0,Jack Edwards,0.374792,0.625164
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029,66288.0,J. Carole Keahey,90069.0,"William L. """"Bill"""" Dickinson",0.423939,0.576029
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699,106935.0,Bill Nichols,0.0,0,0.989699,0.000000
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820,141490.0,Tom Bevill,34531.0,Leonard Wilson,0.803820,0.196174


In [264]:
votes_df.fillna(0, inplace=True)

In [266]:
votes_df.to_csv('HouseVotesbyDistrict_76_18.csv')

## Feature Engineering

In [290]:
votes_df = pd.read_csv('HouseVotesbyDistrict_76_18.csv')

In [291]:
votes_df['winner'] = votes_df['winner'].map(lambda x: re.sub(r'\W+', '', x))

In [293]:
votes_df.sort_values(['district_id', 'year'], inplace=True)

In [294]:
votes_df['L1_winner']= votes_df.winner.shift()

In [295]:
votes_df['L2_winner']= votes_df.winner.shift(2)

In [296]:
votes_df['L3_winner']= votes_df.winner.shift(3)

In [297]:
votes_df['L4_winner']= votes_df.winner.shift(4)

In [298]:
votes_df['L5_winner']= votes_df.winner.shift(5)

In [306]:
votes_df['L6_winner']= votes_df.winner.shift(6)

In [307]:
votes_df['L7_winner']= votes_df.winner.shift(7)

In [308]:
votes_df['L8_winner']= votes_df.winner.shift(8)

In [309]:
votes_df['L9_winner']= votes_df.winner.shift(9)

In [310]:
votes_df['L10_winner']= votes_df.winner.shift(10)

In [303]:
votes_df['dL1_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L1_winner'], 'dL1_winner']= 1

In [314]:
votes_df['incumbent_party'] = votes_df.winner_party.shift()

In [316]:
votes_df['dL2_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L2_winner'], 'dL2_winner']= 1

In [317]:
votes_df['dL3_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L3_winner'], 'dL3_winner']= 1

In [318]:
votes_df['dL4_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L4_winner'], 'dL4_winner']= 1

In [319]:
votes_df['dL5_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L5_winner'], 'dL5_winner']= 1

In [320]:
votes_df['dL6_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L6_winner'], 'dL6_winner']= 1

In [321]:
votes_df['dL7_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L7_winner'], 'dL7_winner']= 1

In [322]:
votes_df['dL8_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L8_winner'], 'dL8_winner']= 1

In [323]:
votes_df['dL9_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L9_winner'], 'dL9_winner']= 1

In [324]:
votes_df['dL10_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L10_winner'], 'dL10_winner']= 1

In [326]:
votes_df['incumbent_L5_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] + votes_df['dL4_winner'] + votes_df['dL5_winner']

In [327]:
votes_df['incumbent_L4_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] + votes_df['dL4_winner'] 

In [328]:
votes_df['incumbent_L3_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] 

In [329]:
votes_df['incumbent_L2_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner']

In [331]:
votes_df['incumbent_L10_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] + votes_df['dL4_winner'] + votes_df['dL5_winner'] + \
    votes_df['dL6_winner'] + votes_df['dL7_winner'] + votes_df['dL8_winner'] + \
    votes_df['dL9_winner'] + votes_df['dL10_winner']

In [333]:
votes_df['rep_wins'] = 0
votes_df.loc[votes_df['winner_party'] == 'republican', 'rep_wins'] = 1 

In [335]:
votes_df['dem_wins'] = 0
votes_df.loc[votes_df['winner_party'] == 'democrat', 'dem_wins'] = 1 

In [336]:
votes_df['rep_L5_wins'] = votes_df.rep_wins.rolling(window=5).sum()

In [342]:
votes_df['rep_L4_wins'] = votes_df.rep_wins.rolling(window=4).sum()

In [343]:
votes_df['rep_L3_wins'] = votes_df.rep_wins.rolling(window=3).sum()

In [344]:
votes_df['rep_L2_wins'] = votes_df.rep_wins.rolling(window=2).sum()

In [347]:
votes_df['rep_L1_wins'] = 0
votes_df.loc[votes_df['incumbent_party'] == 'republican', 'rep_L1_wins']= 1

In [338]:
votes_df['rep_L10_wins'] = votes_df.rep_wins.rolling(window=10).sum()

In [339]:
votes_df['dem_L10_wins'] = votes_df.dem_wins.rolling(window=10).sum()

In [340]:
votes_df['dem_L5_wins'] = votes_df.dem_wins.rolling(window=5).sum()

In [349]:
votes_df['dem_L4_wins'] = votes_df.dem_wins.rolling(window=4).sum()

In [350]:
votes_df['dem_L3_wins'] = votes_df.dem_wins.rolling(window=3).sum()

In [351]:
votes_df['dem_L2_wins'] = votes_df.dem_wins.rolling(window=2).sum()

In [352]:
votes_df['dem_L1_wins'] = 0
votes_df.loc[votes_df['incumbent_party'] == 'democrat', 'dem_L1_wins']= 1

In [355]:
votes_df['rep_L10_voteshare'] = votes_df.rep_voteshare.rolling(window=10).mean()

In [353]:
votes_df['rep_L5_voteshare'] = votes_df.rep_voteshare.rolling(window=5).mean()

In [356]:
votes_df['rep_L4_voteshare'] = votes_df.rep_voteshare.rolling(window=4).mean()

In [357]:
votes_df['rep_L3_voteshare'] = votes_df.rep_voteshare.rolling(window=3).mean()

In [361]:
votes_df['rep_L2_voteshare'] = votes_df.rep_voteshare.rolling(window=2).mean()

In [362]:
votes_df['rep_L1_voteshare']= votes_df.rep_voteshare.shift()

In [363]:
votes_df['dem_L10_voteshare'] = votes_df.dem_voteshare.rolling(window=10).mean()

In [364]:
votes_df['dem_L5_voteshare'] = votes_df.dem_voteshare.rolling(window=5).mean()

In [365]:
votes_df['dem_L4_voteshare'] = votes_df.dem_voteshare.rolling(window=4).mean()

In [366]:
votes_df['dem_L3_voteshare'] = votes_df.dem_voteshare.rolling(window=3).mean()

In [367]:
votes_df['dem_L2_voteshare'] = votes_df.dem_voteshare.rolling(window=2).mean()

In [368]:
votes_df['dem_L1_voteshare']= votes_df.dem_voteshare.shift()

In [370]:
votes_df['target'] = 0
votes_df.loc[votes_df['winner_party'] == 'democrat', 'target']= 1

In [371]:
votes_df

,Unnamed: 0,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,...,rep_L3_voteshare,rep_L2_voteshare,rep_L1_voteshare,dem_L10_voteshare,dem_L5_voteshare,dem_L4_voteshare,dem_L3_voteshare,dem_L2_voteshare,dem_L1_voteshare,target
0,0,1976-AK-00,83722,118208,1976,Alaska,DonYoung,republican,False,AK-00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,435,1978-AK-00,68811,124187,1978,Alaska,DonYoung,republican,False,AK-00,...,NaN,0.631176,0.708260,NaN,NaN,NaN,NaN,0.366784,0.289270,0
2,870,1980-AK-00,114089,154618,1980,Alaska,DonYoung,republican,False,AK-00,...,0.666743,0.645984,0.554092,NaN,NaN,NaN,0.330588,0.351248,0.444298,0
3,1305,1982-AK-00,128274,181084,1982,Alaska,DonYoung,republican,False,AK-00,...,0.666779,0.723122,0.737877,NaN,NaN,0.319746,0.329905,0.272709,0.258198,0
4,1740,1984-AK-00,113582,206437,1984,Alaska,DonYoung,republican,False,AK-00,...,0.665482,0.629285,0.708367,NaN,0.339166,0.351640,0.320754,0.352032,0.287220,0
5,2175,1986-AK-00,101799,180277,1986,Alaska,DonYoung,republican,False,AK-00,...,0.615483,0.569040,0.550202,NaN,0.363467,0.343259,0.371613,0.413809,0.416844,0
6,2610,1988-AK-00,120595,192955,1988,Alaska,DonYoung,republican,False,AK-00,...,0.587690,0.606434,0.587879,NaN,0.349112,0.371841,0.400048,0.391650,0.410773,0
7,3045,1990-AK-00,99003,191647,1990,Alaska,DonYoung,republican,False,AK-00,...,0.576486,0.570790,0.624990,NaN,0.393146,0.419627,0.420555,0.425446,0.372527,0
8,3480,1992-AK-00,111849,239116,1992,Alaska,DonYoung,republican,False,AK-00,...,0.536447,0.492175,0.516590,NaN,0.429302,0.432417,0.439631,0.473183,0.478364,0
9,3915,1994-AK-00,118537,208240,1994,Alaska,DonYoung,republican,False,AK-00,...,0.517861,0.518497,0.467760,0.385504,0.431843,0.437110,0.458638,0.448775,0.468003,0


In [372]:
votes_df_2012_2018 = votes_df[votes_df.year > 2010]
votes_df_2012_2018

,Unnamed: 0,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,...,rep_L3_voteshare,rep_L2_voteshare,rep_L1_voteshare,dem_L10_voteshare,dem_L5_voteshare,dem_L4_voteshare,dem_L3_voteshare,dem_L2_voteshare,dem_L1_voteshare,target
18,7813,2012-AK-00,185296,289804,2012,Alaska,DonYoung,republican,False,AK-00,...,0.627413,0.690409,0.689579,0.337937,0.342113,0.362218,0.347010,0.295641,0.305133,0
19,8248,2014-AK-00,142572,279741,2014,Alaska,DonYoung,republican,False,AK-00,...,0.655527,0.638501,0.691240,0.335949,0.371708,0.362675,0.333651,0.347910,0.286149,0
20,8683,2016-AK-00,155088,308198,2016,Alaska,DonYoung,republican,False,AK-00,...,0.627765,0.596028,0.585763,0.333620,0.362184,0.340293,0.352013,0.384946,0.409672,0
21,9118,2018-AK-00,149779,282166,2018,Alaska,DonYoung,republican,False,AK-00,...,0.574291,0.568555,0.606292,0.342878,0.365229,0.380253,0.411621,0.412595,0.360220,0
40,7814,2012-AL-01,196374,200676,2012,Alabama,JoBonner,republican,False,AL-01,...,0.929036,0.902201,0.825839,0.172897,0.137269,0.079549,0.000000,0.000000,0.000000,0
41,8249,2014-AL-01,103758,152234,2014,Alabama,BradleyByrne,republican,False,AL-01,...,0.828657,0.830066,0.978562,0.171946,0.127065,0.079283,0.105710,0.158565,0.000000,0
42,8684,2016-AL-01,208083,215893,2016,Alabama,BradleyByrne,republican,False,AL-01,...,0.874652,0.822697,0.681569,0.138127,0.063426,0.079283,0.105710,0.158565,0.317130,0
43,9119,2018-AL-01,153228,242617,2018,Alabama,BradleyByrne,republican,False,AL-01,...,0.758986,0.797694,0.963825,0.174903,0.136979,0.171224,0.228298,0.183882,0.000000,0
62,7815,2012-AL-02,180591,283953,2012,Alabama,MarthaRoby,republican,False,AL-02,...,0.547275,0.572859,0.509729,0.345410,0.388443,0.414407,0.451100,0.425483,0.487906,0
63,8250,2014-AL-02,113103,167952,2014,Alabama,MarthaRoby,republican,False,AL-02,...,0.606381,0.654707,0.635989,0.351562,0.396654,0.419735,0.392202,0.344350,0.363060,0


In [375]:
votes_df_2012_2018.target.value_counts()

0    918
1    822
Name: target, dtype: int64

In [376]:
votes_df_2012_2018.to_csv('HouseVotesFeatures_12_18.csv')

## Notes

In [70]:
df[df.party == 'democrat'].groupby(['district_id','year']).candidatevotes.sum()

district_id  year
AK-00        1976     34194
             1978     55176
             1980     39922
             1982     52011
             1984     86052
             1986     74053
             1988     71881
             1990     91677
             1992    102378
             1994     68172
             1996     85114
             1998     77232
             2000     45372
             2002     39357
             2004     67074
             2006     93879
             2008    142560
             2010     77606
             2012     82927
             2014    114602
             2016    111019
             2018    131199
AL-01        1976     58906
             1978     40450
             1982     54315
             1984     98455
             1988     77670
             1992     78742
             1994     50227
             1996     69470
                      ...  
WV-04        1976     73626
             1978     70035
             1980    117595
             1982     91184
  

In [100]:
def func(x):
     return pd.Series(dict(year = x['year'].mean(),
                           state = x['state'].unique()[0], 
                           special = x['special'].mean(), 
                           party = x['party'].unique()[0],
                           writein = x['writein'].mean(),
                           candidatevotes = x['candidatevotes'].max(), 
                           totalvotes = x['totalvotes'].mean(),
                           district_id = x['district_id'].unique()[0],
                           voteshare = x['voteshare'].max(),
                           yr_district_id = x['yr_district_id'].unique()[0]))

In [49]:
fc_ga = fc_all.groupby(['recipient_parent_name1']).apply(func)